In [1]:
# libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import numpy as np
import time
import lightgbm as lgb
import datetime
import time

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold,GroupKFold,StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
ID_51_2020_08 = pd.read_csv(r'D:\留德十年\TUD\ADP\Data\2020/ID_51_2020_08.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')
ID_85_2020 = pd.read_csv(r'D:\留德十年\TUD\ADP\Data\2020/TagID_085_2020.csv',dtype= {'CreateDateTime': str, 'TagValue': np.float64, 'TagId': np.int32},parse_dates=['CreateDateTime'],index_col='Unnamed: 0')

In [35]:
###目标变量
TREE_1 = pd.DataFrame()
TREE_1 = ID_51_2020_08[(ID_51_2020_08['CreateDateTime']>'2020-08-05 10:00:00')&(ID_51_2020_08['CreateDateTime']<'2020-08-05 11:00:00')]
TREE_1 = TREE_1.copy()
TREE_1.rename(columns = {'TagValue':'TagValue_51'}, inplace=True)


###特征合并函数
def zusammen(data,TREE_1):
    data = data[(data['CreateDateTime']>'2020-08-05 10:00:00')&(data['CreateDateTime']<'2020-08-05 11:00:00')]
    data = data.drop('TagId',axis = 1) 
    TREE_1 = TREE_1.set_index('CreateDateTime').join(data.set_index('CreateDateTime'),lsuffix = '_l', rsuffix = '_r',how = 'outer')
    TREE_1 = TREE_1.reset_index('CreateDateTime')
    return TREE_1


###合并特征
TREE_1 = zusammen(ID_85_2020,TREE_1)
TREE_1 = TREE_1.copy()
TREE_1.rename(columns = {'TagValue':'TagValue_85'}, inplace=True)
TREE_1['timestamp'] = TREE_1['CreateDateTime'].values.astype(np.int64) // 10 ** 6
TREE_1

,CreateDateTime,TagValue_51,TagId,TagValue_85,timestamp
0,2020-08-05 10:00:00.375,-387.731410,51.0,NaN,1596621600375
1,2020-08-05 10:00:00.570,-388.310210,51.0,NaN,1596621600570
2,2020-08-05 10:00:00.776,-390.046300,51.0,NaN,1596621600776
3,2020-08-05 10:00:01.180,-391.203700,51.0,NaN,1596621601180
4,2020-08-05 10:00:01.791,-391.782410,51.0,NaN,1596621601791
...,...,...,...,...,...
12293,2020-08-05 10:53:34.796,0.000000,51.0,NaN,1596624814796
12294,2020-08-05 10:53:35.410,-0.578735,51.0,NaN,1596624815410
12295,2020-08-05 10:53:36.225,0.000000,51.0,NaN,1596624816225
12296,2020-08-05 10:53:36.430,-0.578735,51.0,NaN,1596624816430


In [36]:
TREE_1.isnull().sum()

CreateDateTime        0
TagValue_51           3
TagId                 3
TagValue_85       12293
timestamp             0
dtype: int64

In [37]:
np.where(np.isnan(TREE_1['TagValue_51']))

(array([ 1600,  8736, 10442], dtype=int64),)

In [38]:
TREE_1.loc[1598:1602]

,CreateDateTime,TagValue_51,TagId,TagValue_85,timestamp
1598,2020-08-05 10:07:58.788,-248.26390,51.0,NaN,1596622078788
1599,2020-08-05 10:07:59.194,-250.00000,51.0,NaN,1596622079194
1600,2020-08-05 10:07:59.608,NaN,NaN,0.0,1596622079608
1601,2020-08-05 10:07:59.609,-251.73610,51.0,NaN,1596622079609
1602,2020-08-05 10:07:59.817,-251.15739,51.0,NaN,1596622079817


In [39]:
TREE_1.loc[8733:8739]

,CreateDateTime,TagValue_51,TagId,TagValue_85,timestamp
8733,2020-08-05 10:37:49.293,-311.92130,51.0,NaN,1596623869293
8734,2020-08-05 10:37:49.502,-310.76389,51.0,NaN,1596623869502
8735,2020-08-05 10:37:49.698,-313.65741,51.0,NaN,1596623869698
8736,2020-08-05 10:37:49.897,NaN,NaN,1.0,1596623869897
8737,2020-08-05 10:37:49.898,-319.44449,51.0,NaN,1596623869898
8738,2020-08-05 10:37:50.116,-267.36111,51.0,NaN,1596623870116
8739,2020-08-05 10:37:50.312,-168.40280,51.0,NaN,1596623870312


In [40]:
s = TREE_1.set_index('timestamp')['TagValue_51']
s

timestamp
1596621600375   -387.731410
1596621600570   -388.310210
1596621600776   -390.046300
1596621601180   -391.203700
1596621601791   -391.782410
                    ...    
1596624814796      0.000000
1596624815410     -0.578735
1596624816225      0.000000
1596624816430     -0.578735
1596624816633      0.000000
Name: TagValue_51, Length: 12298, dtype: float64

In [41]:
s[1598:1602]

timestamp
1596622078788   -248.2639
1596622079194   -250.0000
1596622079608         NaN
1596622079609   -251.7361
Name: TagValue_51, dtype: float64

In [42]:
s[8733:8739]

timestamp
1596623869293   -311.92130
1596623869502   -310.76389
1596623869698   -313.65741
1596623869897          NaN
1596623869898   -319.44449
1596623870116   -267.36111
Name: TagValue_51, dtype: float64

In [43]:
s = s.interpolate(method='index',inpalce=True)

In [44]:
s[1598:1602]

timestamp
1596622078788   -248.263900
1596622079194   -250.000000
1596622079608   -251.731917
1596622079609   -251.736100
Name: TagValue_51, dtype: float64

In [45]:
s[8733:8739]

timestamp
1596623869293   -311.921300
1596623869502   -310.763890
1596623869698   -313.657410
1596623869897   -319.415555
1596623869898   -319.444490
1596623870116   -267.361110
Name: TagValue_51, dtype: float64

In [46]:
s

timestamp
1596621600375   -387.731410
1596621600570   -388.310210
1596621600776   -390.046300
1596621601180   -391.203700
1596621601791   -391.782410
                    ...    
1596624814796      0.000000
1596624815410     -0.578735
1596624816225      0.000000
1596624816430     -0.578735
1596624816633      0.000000
Name: TagValue_51, Length: 12298, dtype: float64

In [47]:
# right = s
# left = TREE_1
TREE_1 = pd.merge(left=TREE_1, right=s, on=['timestamp'])
TREE_1

,CreateDateTime,TagValue_51_x,TagId,TagValue_85,timestamp,TagValue_51_y
0,2020-08-05 10:00:00.375,-387.731410,51.0,NaN,1596621600375,-387.731410
1,2020-08-05 10:00:00.570,-388.310210,51.0,NaN,1596621600570,-388.310210
2,2020-08-05 10:00:00.776,-390.046300,51.0,NaN,1596621600776,-390.046300
3,2020-08-05 10:00:01.180,-391.203700,51.0,NaN,1596621601180,-391.203700
4,2020-08-05 10:00:01.791,-391.782410,51.0,NaN,1596621601791,-391.782410
...,...,...,...,...,...,...
12293,2020-08-05 10:53:34.796,0.000000,51.0,NaN,1596624814796,0.000000
12294,2020-08-05 10:53:35.410,-0.578735,51.0,NaN,1596624815410,-0.578735
12295,2020-08-05 10:53:36.225,0.000000,51.0,NaN,1596624816225,0.000000
12296,2020-08-05 10:53:36.430,-0.578735,51.0,NaN,1596624816430,-0.578735


In [48]:
TREE_1.loc[8733:8739]

,CreateDateTime,TagValue_51_x,TagId,TagValue_85,timestamp,TagValue_51_y
8733,2020-08-05 10:37:49.293,-311.92130,51.0,NaN,1596623869293,-311.921300
8734,2020-08-05 10:37:49.502,-310.76389,51.0,NaN,1596623869502,-310.763890
8735,2020-08-05 10:37:49.698,-313.65741,51.0,NaN,1596623869698,-313.657410
8736,2020-08-05 10:37:49.897,NaN,NaN,1.0,1596623869897,-319.415555
8737,2020-08-05 10:37:49.898,-319.44449,51.0,NaN,1596623869898,-319.444490
8738,2020-08-05 10:37:50.116,-267.36111,51.0,NaN,1596623870116,-267.361110
8739,2020-08-05 10:37:50.312,-168.40280,51.0,NaN,1596623870312,-168.402800


In [49]:
TREE_1['TagValue_51_x'] = TREE_1['TagValue_51_y']
TREE_1.loc[8733:8739]

,CreateDateTime,TagValue_51_x,TagId,TagValue_85,timestamp,TagValue_51_y
8733,2020-08-05 10:37:49.293,-311.921300,51.0,NaN,1596623869293,-311.921300
8734,2020-08-05 10:37:49.502,-310.763890,51.0,NaN,1596623869502,-310.763890
8735,2020-08-05 10:37:49.698,-313.657410,51.0,NaN,1596623869698,-313.657410
8736,2020-08-05 10:37:49.897,-319.415555,NaN,1.0,1596623869897,-319.415555
8737,2020-08-05 10:37:49.898,-319.444490,51.0,NaN,1596623869898,-319.444490
8738,2020-08-05 10:37:50.116,-267.361110,51.0,NaN,1596623870116,-267.361110
8739,2020-08-05 10:37:50.312,-168.402800,51.0,NaN,1596623870312,-168.402800


In [50]:
TREE_1['TagValue_51_x'] = TREE_1['TagValue_51_y']
TREE_1 = TREE_1.drop('TagValue_51_y',axis=1)
TREE_1 = TREE_1.drop('timestamp',axis=1)
TREE_1 = TREE_1.copy()
TREE_1.rename(columns = {'TagValue_51_x':'TagValue_51'}, inplace=True)
TREE_1

,CreateDateTime,TagValue_51,TagId,TagValue_85
0,2020-08-05 10:00:00.375,-387.731410,51.0,NaN
1,2020-08-05 10:00:00.570,-388.310210,51.0,NaN
2,2020-08-05 10:00:00.776,-390.046300,51.0,NaN
3,2020-08-05 10:00:01.180,-391.203700,51.0,NaN
4,2020-08-05 10:00:01.791,-391.782410,51.0,NaN
...,...,...,...,...
12293,2020-08-05 10:53:34.796,0.000000,51.0,NaN
12294,2020-08-05 10:53:35.410,-0.578735,51.0,NaN
12295,2020-08-05 10:53:36.225,0.000000,51.0,NaN
12296,2020-08-05 10:53:36.430,-0.578735,51.0,NaN
